In [1]:
import pandas as pd
import sqlite3
import glob
import os

In [2]:
eventTypeTableFilenames = [fn for fn in glob.glob('event_*.csv') 
                           if not fn == "event_map_type.csv" and not fn == "event_object.csv"]
objectTypeTableFilenames =  [fn for fn in glob.glob('object_*.csv') 
                             if not fn == "object_map_type.csv" and not fn == "object_object.csv"]

TABLES = dict()

TABLES["event"] = pd.read_csv("event.csv", sep=";")
TABLES["event_map_type"] = pd.read_csv("event_map_type.csv", sep=";")
TABLES["event_object"] = pd.read_csv("event_object.csv", sep=";")
TABLES["object"] = pd.read_csv("object.csv", sep=";")
TABLES["object_object"] = pd.read_csv("object_object.csv", sep=";")
TABLES["object_map_type"] = pd.read_csv("object_map_type.csv", sep=";")

for fn in eventTypeTableFilenames:
    table_name = fn.split(".")[0]
    table = pd.read_csv(fn, sep=";")
    TABLES[table_name] = table
    
for fn in objectTypeTableFilenames:
    table_name = fn.split(".")[0]
    table = pd.read_csv(fn, sep=";")
    TABLES[table_name] = table

In [3]:
sql_path = "CargoPickup.sqlite"
if os.path.exists(sql_path):
    os.remove(sql_path)

In [4]:
conn = sqlite3.connect(sql_path)
for tn, df in TABLES.items():
    df.to_sql(tn, conn, index=False)
conn.close()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [5]:
# Connect to your SQLite database
conn = sqlite3.connect("CargoPickup.sqlite")
cursor = conn.cursor()

# Create a new table without duplicates
cursor.execute('''CREATE TABLE IF NOT EXISTS object_object_no_duplicates AS
                  SELECT DISTINCT * FROM object_object''')

# Drop the original table
cursor.execute('DROP TABLE IF EXISTS object_object')

# Rename the new table to the original table name
cursor.execute('ALTER TABLE object_object_no_duplicates RENAME TO object_object')

# Commit changes and close the connection
conn.commit()
conn.close()

In [6]:
import pm4py

ocel = pm4py.read_ocel2_sqlite("CargoPickup.sqlite")
ocel

/opt/anaconda3/lib/python3.8/site-packages/pm4py/objects/ocel/importer/sqlite/variants/ocel20.py:70: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_14_primary_key_object_event_map_type_tables', 'const_15_primary_key_object_event_tables', 'const_16_primary_key_event_object_table', 'const_17_primary_key_object_object_table', 'const_18_primary_key_event_type_spec_tables', 'const_19_foreign_key_event', 'const_20_foreign_key_object', 'const_21_foreign_key_event_object', 'const_22_foreign_key_object_object', 'const_23_foreign_key_event_type_specific', 'const_24_foreign_key_object_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
/opt/anaconda3/lib/python3.8/site-packages/pm4py/objects/ocel/importer/sqlite/variants/ocel20.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

Object-Centric Event Log (number of events: 1892, number of objects: 250, number of activities: 8, number of object types: 3, events-objects relationships: 1992)
Activities occurrences: {'Weigh the Empty Truck': 256, 'Weigh the Loaded Truck': 256, 'Input the Tally Sheet': 256, 'Evaluate the Truck Exit': 256, 'Load Truck': 256, 'Assign Trucks': 256, 'Print the Weighing Ticket': 256, 'Lodge Pickup Plan': 100}
Object types occurrences (number of objects): {'Cargo': 100, 'Pickup Plan': 100, 'Truck': 50}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.